In [1]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
%matplotlib inline

from make_map import make_map

from file_util import load_matlab_data
# from channel_overlap import channel_overlap
from plot import plot_discharge
from metrics import *
from stratigraphy import sedimentograph


import cPickle as pickle

In [2]:
TH_TOPO = -0.5
TH_ANGLE = 75
TH_VEL = 0.3 # use the threshold value for mud deposition (U_dep_mud)

fbase = 'baseF25data'
froot = '../JGR2016_data/'
sroot = 'mapfiles/'

In [3]:
for fnum in range(4005,4006):

    fname = fbase + str(fnum)
    data = load_matlab_data(froot, fname)

    topo = data['eta']
    uw = data['uw']

    # clip to make the file smaller
    topo_clip = topo[3:-5,5:-10]
    vel_clip = uw[3:-5,5:-10]

    mapfile = make_map(topo_clip,
                       vel_clip,
                       topo_threshold = TH_TOPO,
                       angle_threshold = TH_ANGLE,
                       velocity_threshold = TH_VEL,
                       numviews = 5,
                       save_file = True,
                       sroot = sroot,
                       fname = fname)

    frac_areas = fractional_areas(mapfile)
    EdgeDistMap, histogram = nearest_edge_distance(mapfile)
    D_frac = fractal_dimension(mapfile['centerlinemap'])
    
    island_props = island_properties(mapfile)
    
    seds = sedimentograph(data['strata'])

In [6]:
import numpy as np
import cPickle as pickle

def channel_overlap(filenames):
    '''
    Accepts a list of filenames for mapfiles and
    compares the position of their channels.
    
    Follows same order as the list
    '''

    # first mapfile in list
    f = filenames[0]
    mapfile = pickle.load( open( f, "rb" ) )
    data = mapfile['channelmap']

    # create empty overlap map arrays
    overlapmap = np.zeros(data.shape)
    difference, phi, O_phi, f_R = np.zeros((4, len(filenames,)))

    # create circular mask
    maskmap, area_mask = create_circular_mask(data, data.shape[1]/2., 0, 55, 5)

    # get base image values
    chan_base = data * maskmap
    fw_base = chan_base.sum() / area_mask
    Adry_base = area_mask * (1 - fw_base)
    


    for n,f in enumerate(filenames):

        mapfile = pickle.load( open( f, "rb" ) )
        data = mapfile['channelmap']

        chan_step = data * maskmap
        fw_step = chan_step.sum() / area_mask

        difference[n] = np.abs(chan_base - chan_step).sum()

        phi[n] = fw_base * (1 - fw_step) + (1 - fw_base) * fw_step
        O_phi[n] = 1 - difference[n] / (area_mask * phi[n])

        overlapmap = overlapmap + chan_step
        drymap = np.maximum(0, 1 - overlapmap) * maskmap
        f_R[n] = 1 - drymap.sum() / Adry_base


    return overlapmap, difference, phi, O_phi, f_R
    
    
    
def create_circular_mask(data, x0, y0, r1, r2):
    
    n = max(data.shape)
    y,x = np.ogrid[-y0:n-y0, -x0:n-x0]

    mask = (x*x + y*y <= r1*r1) & (x*x + y*y >= r2*r2)

    array = np.zeros((n, n))
    array[mask] = 1
    maskmap = array[:data.shape[0], :data.shape[1]] > 0
    area_mask = maskmap.sum()
    
    return maskmap, float(area_mask)

In [8]:
filenames = [sroot + fbase + str(i) + '.p' for i in range(4005,4010)]
overlapmap, difference, phi, O_phi, f_R = channel_overlap(filenames)